In [1]:
import tensorflow as tf
import keras
from datetime import datetime

%load_ext tensorboard

In [2]:
class SimpleModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.a = tf.Variable(5.0, trainable=True, name="train_me")
        self.b = tf.Variable(5.0, trainable=False, name="dont_train_me")
    def __call__(self, x):
        return x * self.a + x * self.b

mod1=SimpleModule(name="mod1")
mod1(5)

<tf.Tensor: shape=(), dtype=float32, numpy=50.0>

In [3]:
mod1.trainable_variables

(<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>,)

In [4]:
mod1.non_trainable_variables

(<tf.Variable 'dont_train_me:0' shape=() dtype=float32, numpy=5.0>,)

In [5]:
class Dense(tf.Module):
    def __init__(self, in_features,out_features, name=None):
        super().__init__(name=name)
        self.w = tf.Variable(tf.random.normal(shape=(in_features, out_features)), name="w")
        self.b = tf.Variable(tf.random.normal(shape=(1, out_features)), name="b")
    def __call__(self,x):
        y= tf.matmul(x,self.w)+self.b
        return tf.nn.softmax(y)


In [6]:
class Sequential(tf.Module):
    def __init__(self, name=None):
        super().__init__(name)
        self.dense1 = Dense(3,3,name='dense1')
        self.dense2 = Dense(3,2,name='dense2')
    def __call__(self,x):
        x = self.dense1(x)
        return self.dense2(x)

x= tf.constant([ [1.,2.,3.]])
seq_mod = Sequential(name='seq_mod')
seq_mod(x)

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.01232626, 0.9876737 ]], dtype=float32)>

In [7]:
seq_mod.submodules

(<__main__.Dense at 0x14abb4e80>, <__main__.Dense at 0x14ab832e0>)

In [8]:
seq_mod.variables

(<tf.Variable 'b:0' shape=(1, 3) dtype=float32, numpy=array([[-0.9973821,  2.209878 ,  1.1785421]], dtype=float32)>,
 <tf.Variable 'w:0' shape=(3, 3) dtype=float32, numpy=
 array([[-0.15010975, -1.5529915 ,  1.6065135 ],
        [ 0.34934926,  0.20236862,  1.4073561 ],
        [-0.60944146, -1.4005055 ,  1.7258562 ]], dtype=float32)>,
 <tf.Variable 'b:0' shape=(1, 2) dtype=float32, numpy=array([[-1.1561724,  1.5592358]], dtype=float32)>,
 <tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
 array([[ 1.9516275 ,  0.47359705],
        [ 0.82912385, -0.3845102 ],
        [ 0.7749526 ,  2.4431744 ]], dtype=float32)>)

In [9]:
chkp_path='my_checkpoint'
checkpoint1 = tf.train.Checkpoint(model=seq_mod)
checkpoint1.write(chkp_path)

'my_checkpoint'

In [10]:
tf.train.list_variables(chkp_path)

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('model/dense1/b/.ATTRIBUTES/VARIABLE_VALUE', [1, 3]),
 ('model/dense1/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 3]),
 ('model/dense2/b/.ATTRIBUTES/VARIABLE_VALUE', [1, 2]),
 ('model/dense2/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 2])]

In [11]:
new_model = Sequential()
new_chkpoint = tf.train.Checkpoint(model=new_model)
new_chkpoint.restore('my_checkpoint')
new_model.variables

(<tf.Variable 'b:0' shape=(1, 3) dtype=float32, numpy=array([[-0.9973821,  2.209878 ,  1.1785421]], dtype=float32)>,
 <tf.Variable 'w:0' shape=(3, 3) dtype=float32, numpy=
 array([[-0.15010975, -1.5529915 ,  1.6065135 ],
        [ 0.34934926,  0.20236862,  1.4073561 ],
        [-0.60944146, -1.4005055 ,  1.7258562 ]], dtype=float32)>,
 <tf.Variable 'b:0' shape=(1, 2) dtype=float32, numpy=array([[-1.1561724,  1.5592358]], dtype=float32)>,
 <tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
 array([[ 1.9516275 ,  0.47359705],
        [ 0.82912385, -0.3845102 ],
        [ 0.7749526 ,  2.4431744 ]], dtype=float32)>)

In [12]:
new_model(x)

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.01232626, 0.9876737 ]], dtype=float32)>

In [13]:
tf.saved_model.save(seq_mod, 'model1')

INFO:tensorflow:Assets written to: model1/assets


In [14]:
mod2=tf.saved_model.load('model1')

In [15]:
class MyDense(tf.keras.layers.Layer):
    def __init__(self, in_features, out_features, **kwargs):
        super().__init__(**kwargs)
        self.w = tf.Variable( tf.random.normal(shape=[in_features,out_features]), name="w")
        self.b = tf.Variable(tf.zeros(shape=[1,out_features]), name="b")
    def call(self, x):
            x = tf.matmul(x, self.w) + self.b
            return tf.nn.softmax(x)

simple_layer = MyDense(name="simple", in_features=3, out_features=3)
simple_layer(x)

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.53348   , 0.46411914, 0.00240087]], dtype=float32)>

In [16]:
class FlexibleDense(tf.keras.layers.Layer):
    def __init__(self, out_features, **kwargs):
        super().__init__(**kwargs)
        self.out_features = out_features
    def build(self, input_shape):
        self.w = tf.Variable( tf.random.normal(shape=[input_shape[-1],self.out_features]), name="w")
        self.b = tf.Variable(tf.zeros(shape=[1,self.out_features]), name="b")
    def call(self, x):
            x = tf.matmul(x, self.w) + self.b
            return tf.nn.softmax(x)
flex_layer = FlexibleDense(name="simple", out_features=3)
flex_layer(x)

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.00676718, 0.00218155, 0.9910513 ]], dtype=float32)>

In [17]:
flex_layer.variables

[<tf.Variable 'simple/w:0' shape=(3, 3) dtype=float32, numpy=
 array([[-0.38703758,  1.190179  ,  0.9455425 ],
        [-0.5336971 ,  1.0311134 ,  1.0871806 ],
        [ 0.10656951, -1.8397255 ,  0.24401845]], dtype=float32)>,
 <tf.Variable 'simple/b:0' shape=(1, 3) dtype=float32, numpy=array([[0., 0., 0.]], dtype=float32)>]

In [18]:
x1=[[1.,2.]]
# flex_layer(x1)

In [19]:
class MySequentialModel(tf.keras.Model):
    def __init__(self, name=None, **kwargs):
        super().__init__(**kwargs)
        self.dense1 = FlexibleDense(out_features=3)
        self.dense2 = FlexibleDense(out_features=2)
        
    def call(self, x):
        x = self.dense1(x)
        return self.dense2(x)

my_sequential_model=MySequentialModel(name='MySequentialModel')

In [21]:
my_sequential_model.variables

[]

In [22]:
x=tf.constant([[1.,2.,3.]])
my_sequential_model(x)

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.70187306, 0.2981269 ]], dtype=float32)>

In [23]:
my_sequential_model.variables

[<tf.Variable 'my_sequential_model/flexible_dense/w:0' shape=(3, 3) dtype=float32, numpy=
 array([[-0.40164053,  0.5788735 , -0.89743245],
        [-0.46564883,  1.0050181 ,  0.27754086],
        [ 1.5531322 , -0.00512442,  0.21605258]], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense/b:0' shape=(1, 3) dtype=float32, numpy=array([[0., 0., 0.]], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_1/w:0' shape=(3, 2) dtype=float32, numpy=
 array([[ 1.5544016 , -0.05727765],
        [-0.45261735, -0.0070276 ],
        [-1.7785462 ,  0.28308097]], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_1/b:0' shape=(1, 2) dtype=float32, numpy=array([[0., 0.]], dtype=float32)>]

In [24]:
my_sequential_model.submodules

(<__main__.FlexibleDense at 0x14d403f10>,
 <__main__.FlexibleDense at 0x14d40b4f0>)

In [25]:
my_sequential_model.summary()

Model: "my_sequential_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flexible_dense (FlexibleDe  multiple                  12        
 nse)                                                            
                                                                 
 flexible_dense_1 (Flexible  multiple                  8         
 Dense)                                                          
                                                                 
Total params: 20 (80.00 Byte)
Trainable params: 20 (80.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
inputs = tf.keras.Input(shape=[1,3])
x = FlexibleDense(3)(inputs)
output = FlexibleDense(2)(x)
my_functional_model = tf.keras.Model(inputs=inputs, outputs=output)

In [28]:
my_functional_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1, 3)]            0         
                                                                 
 flexible_dense_4 (Flexible  (None, 1, 3)              12        
 Dense)                                                          
                                                                 
 flexible_dense_5 (Flexible  (None, 1, 2)              8         
 Dense)                                                          
                                                                 
Total params: 20 (80.00 Byte)
Trainable params: 20 (80.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
my_sequential_model.save('mykerasmodel')

INFO:tensorflow:Assets written to: mykerasmodel/assets


INFO:tensorflow:Assets written to: mykerasmodel/assets


In [34]:
reconstructed_model = tf.keras.models.load_model('mykerasmodel')
reconstructed_model(tf.constant([[2.0, 2.0, 2.0],[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[0.45493528, 0.5450648 ],
       [0.45493528, 0.5450648 ]], dtype=float32)>